<a href="https://colab.research.google.com/github/woghd8503/kernel-academy-web-crawling/blob/main/bsoup_fastcampus_middle_%ED%95%99%EC%83%9D%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 웹 데이터 수집 중급: BeautifulSoup과 Requests로 Quotes to Scrape 크롤링 quote️ (중급)

 지난 시간에는 BeautifulSoup의 기본 사용법을 익혔습니다. 이번 중급 과정에서는 실제 웹사이트인 'Quotes to Scrape' ([http://quotes.toscrape.com/](http://quotes.toscrape.com/)) 사이트의 데이터를 수집해 보겠습니다.

**학습 목표:**
1. `requests` 라이브러리를 사용하여 웹 페이지의 HTML 내용 가져오기
2. 웹사이트에서 명언, 저자, 관련 태그 정보 추출하기
3. 여러 페이지에 걸친 데이터 수집 방법 (페이지네이션 처리 기초) 이해하기
4. CSS 선택자를 활용한 요소 검색 맛보기
5. 수집한 데이터를 Pandas DataFrame으로 간단히 변환하기

**주의:** 웹 스크래핑 시에는 항상 해당 웹사이트의 이용 약관 및 `robots.txt` 파일을 확인하여 서버에 부담을 주지 않는 범위 내에서 합법적으로 데이터를 수집해야 합니다. 'Quotes to Scrape'는 학습 목적으로 스크래핑이 허용된 사이트입니다.

---

## 1. 라이브러리 준비 및 첫 페이지 HTML 가져오기 🌐

웹 페이지 내용을 가져오기 위해 `requests` 라이브러리가 필요합니다. BeautifulSoup은 HTML을 *파싱*하는 도구이지, 웹에서 직접 HTML을 *가져오는* 기능은 없습니다.

1.  `requests`와 `beautifulsoup4`, 그리고 데이터 처리를 위한 `pandas`를 설치합니다. (Colab에는 대부분 미리 설치되어 있습니다.)
2.  필요한 라이브러리들을 import 합니다.
3.  `requests.get(URL)` 함수를 사용하여 'Quotes to Scrape' 사이트의 HTML을 가져옵니다.
4.  가져온 HTML을 `BeautifulSoup` 객체로 만듭니다.
5.  `urllib.parse.urljoin`은 상대 경로 URL을 절대 경로로 만들기 위해 사용됩니다.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin # URL을 조합하기 위해 import 합니다.

In [ ]:
# 스크래핑할 기본 URL
base_url = 'http://quotes.toscrape.com/'

In [ ]:
a = '안녕하세요?'

In [ ]:
# 첫 페이지 HTML 가져오기
response = requests.get(base_url)
# request -> 보내는것  , response -> 받는것 (리턴값)
# get 방식 (url 로 직접 접근 방식 (리턴이 페이지로 넘어옴)) ,
# post 방식 (회원 가입, 폼, 입력할때 정보만 전달하는 방식 (리턴이 필요 없는))
response

<Response [200]>

In [ ]:
# 요청이 성공했는지 확인 (상태 코드 200이면 성공)
if response.status_code == 200:
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    print(f"페이지 제목: {soup.title.string}")
    print(f"첫 페이지 로드 성공!")
else:
    print(f"페이지 로드 실패: 상태 코드 {response.status_code}")
    # 실패 시 이후 코드 실행을 막기 위해 soup을 None으로 설정하거나 예외 발생 처리 가능
    soup = None

페이지 제목: Quotes to Scrape
첫 페이지 로드 성공!


In [ ]:
print(soup.prettify())

---
## 2. 첫 페이지에서 명언, 저자, 태그 추출하기 ✍️

'Quotes to Scrape' 사이트를 보면 각 명언은 `div` 태그 중 `class`가 `quote`인 요소 안에 들어있습니다. 이 구조를 파악하여 정보를 추출합니다.

* **명언(Quote)**: `div.quote` 내부의 `span.text` 요소
* **저자(Author)**: `div.quote` 내부의 `small.author` 요소
* **태그(Tags)**: `div.quote` 내부의 `div.tags` 안에 있는 `a.tag` 요소들

먼저 `find_all()`을 사용해 모든 명언이 담긴 `div.quote` 요소들을 찾고, 각 요소 내부에서 명언, 저자, 태그를 추출합니다.

In [ ]:
# 모든 'div' 태그 중 class가 'quote'인 요소들을 찾습니다.
quote_divs = soup.find_all('div', class_='quote')

print(f"첫 페이지에서 {len(quote_divs)}개의 명언을 찾았습니다.\n")
quotes_data = [] # 추출한 데이터를 저장할 리스트

첫 페이지에서 10개의 명언을 찾았습니다.



In [ ]:
# 명언 추출 for문을 써서
for quote_div in quote_divs:
    text = quote_div.find('span', class_='text').get_text(strip=True) # 앞 뒤 공백이 있다면, 제거해서 가져온다 - strip = True
    print(text)

In [ ]:
# 저자 추출
for quote_div in quote_divs:
    author = quote_div.find('small', class_='author').get_text()
    print(author)

In [ ]:
# 태그 추출 (여러 개일 수 있음)
for quote_div in quote_divs:
    tags_elements = quote_div.find_all('a' , class_= 'tag')
    tags = [tag.get_text() for tag in tags_elements]
    print(tags)

In [ ]:
for i , quote_info in enumerate(quote_divs):
    text = quote_info.find('span', class_='text').get_text(strip=True) # 앞 뒤 공백이 있다면, 제거해서 가져온다 - strip = True
    author = quote_info.find('small', class_='author').get_text()
    tags_elements = quote_info.find_all('a' , class_= 'tag')
    tags = [tag.get_text() for tag in tags_elements]
    quotes_data.append({
        'text': text,
        'author': author,
        'tags': tags
    })
quotes_data

In [ ]:
# q:  enumerate가 뭔가요?

testlist = [100,1,2,3,4,5,6,7]
for idx,value in enumerate(testlist):
    # enumerate 는 idx, value 같이 리턴
    print(idx,value)

In [ ]:
# 반복문으로 통합해 보기!

---
## 3. CSS 선택자로 요소 찾기 🎯

BeautifulSoup은 `find()`나 `find_all()` 외에도 CSS 선택자(Selector)를 사용하여 요소를 찾는 `select()` 와 `select_one()` 메소드를 제공합니다. CSS 선택자는 HTML 구조를 경로처럼 지정하여 요소를 선택하는 강력한 방법입니다.

* `soup.select('선택자')`: 선택자와 일치하는 모든 요소를 리스트로 반환 ( `find_all`과 유사)
* `soup.select_one('선택자')`: 선택자와 일치하는 첫 번째 요소를 반환 ( `find`와 유사)

예시:
* `div.quote`: class가 `quote`인 `div` 태그
* `span.text`: class가 `text`인 `span` 태그
* `div.tags a.tag`: class가 `tags`인 `div` 태그 내부의 class가 `tag`인 모든 `a` 태그

앞서 `find`와 `find_all`로 했던 작업을 `select`를 사용해서 다시 해봅시다.

In [ ]:
quotes_data_with_selector = []

In [ ]:
# CSS 선택자를 사용하여 모든 'div.quote' 요소들을 찾습니다.
quote_divs_selected = soup.select('div.quote')

In [ ]:
print(f"CSS 선택자로 {len(quote_divs_selected)}개의 명언을 찾았습니다.\n")

CSS 선택자로 10개의 명언을 찾았습니다.



In [ ]:
# 명언 추출 (select_one 사용)
text = quote_div.select_one('span.text').get_text()
text

'“A day without sunshine is like, you know, night.”'

In [ ]:
# 저자 추출
author = quote_div.select_one('small.author').get_text()
author

'Steve Martin'

In [ ]:
# 태그 추출 (select 사용)
tags_elements = quote_div.select('div.tags a.tag')
#tags_elements = tags_elements.select('a.tag')
tags = [tag.get_text() for tag in tags_elements]

quotes_data_with_selector.append({
    'text' : text,
    'author' : author,
    'tags' : tags
})
print(quotes_data_with_selector)

In [ ]:
for i, quote_info in enumerate(quotes_data_with_selector[:2]):
    print(f"--- 선택자 명언 {i+1} ---")
    print(f"내용: {quote_info['text']}")
    print(f"저자: {quote_info['author']}")
    print(f"태그: {', '.join(quote_info['tags'])}\n")

--- 선택자 명언 1 ---
내용: “A day without sunshine is like, you know, night.”
저자: Steve Martin
태그: humor, obvious, simile



In [ ]:
# 통합해보기!

---
## 4. 여러 페이지 데이터 수집 (페이지네이션) 📄➡️📄

많은 웹사이트는 한 페이지에 모든 정보를 보여주지 않고 여러 페이지로 나눕니다. 'Quotes to Scrape'도 마찬가지로 'Next →' 버튼이 있어 다음 페이지로 이동할 수 있습니다.

페이지네이션을 처리하는 일반적인 방법
1.  현재 페이지에서 '다음(Next)' 버튼(또는 링크)을 찾습니다.
2.  '다음' 버튼이 있다면 해당 링크(URL)를 가져옵니다. 링크가 상대 경로일 경우, 기본 URL과 조합하여 절대 경로로 만듭니다. (`urllib.parse.urljoin` 사용)
3.  새 URL로 접속하여 위에서 했던 정보 추출 과정을 반복합니다.
4.  '다음' 버튼이 더 이상 없을 때까지 반복합니다.

여기서는  **최대 3 페이지**까지만 데이터를 수집해보겠습니다.

In [ ]:
all_quotes_data = [] # 모든 페이지의 명언을 저장할 리스트
current_url = base_url
max_pages = 100

In [ ]:
for page_num in range(max_pages):
    print(f"\n{page_num + 1}번째 페이지 스크래핑 중: {current_url}")
    response = requests.get(current_url)

    if response.status_code != 200:
        print(f"\n{page_num + 1}번째 페이지 스크래핑 중 오류: {current_url}")

    soup = BeautifulSoup(response.text , 'html.parser')

    quote_divs = soup.select('div.quote') # 명언 있는지 없는지 검토
    if not quote_divs:
        print('명언이 더이상 없습니다.')

    for quote_div in quote_divs:
        text = quote_div.select_one('span.text').get_text()
        author =  quote_div.select_one('small.author').get_text()
        tags = [tag.get_text() for tag in quote_div.select('div.tags a.tag')]
        all_quotes_data.append({'text' : text , 'author' : author, 'tags' : tags})

    next_li = soup.select_one('li.next')
    if next_li:
        next_page_relative_url = next_li.select_one('a')['href']
        current_url = urljoin(base_url,next_page_relative_url)
        #current_url = base_url + next_page_relative_url

    else:
        print('끝 페이지 입니다')
        break
print(f"\n총 {len(all_quotes_data)}개의 명언을 수집했습니다.")



1번째 페이지 스크래핑 중: http://quotes.toscrape.com/

2번째 페이지 스크래핑 중: http://quotes.toscrape.com/page/2/

3번째 페이지 스크래핑 중: http://quotes.toscrape.com/page/3/

4번째 페이지 스크래핑 중: http://quotes.toscrape.com/page/4/

5번째 페이지 스크래핑 중: http://quotes.toscrape.com/page/5/

6번째 페이지 스크래핑 중: http://quotes.toscrape.com/page/6/

7번째 페이지 스크래핑 중: http://quotes.toscrape.com/page/7/

8번째 페이지 스크래핑 중: http://quotes.toscrape.com/page/8/

9번째 페이지 스크래핑 중: http://quotes.toscrape.com/page/9/

10번째 페이지 스크래핑 중: http://quotes.toscrape.com/page/10/
끝 페이지 입니다

총 100개의 명언을 수집했습니다.


In [ ]:

# 수집된 데이터 중 처음 5개와 마지막 2개 출력
if len(all_quotes_data) > 0:
    print("\n--- 수집된 명언 (일부) ---")
    for i, quote_info in enumerate(all_quotes_data[:5]): # 처음 5개
        print(f"{i+1}. {quote_info['author']}: {quote_info['text'][:30]}... [{', '.join(quote_info['tags'])}]")
    if len(all_quotes_data) > 5:
        print("...")
        for i, quote_info in enumerate(all_quotes_data[-2:]): # 마지막 2개
            print(f"{len(all_quotes_data) - 1 + i}. {quote_info['author']}: {quote_info['text'][:30]}... [{', '.join(quote_info['tags'])}]")


--- 수집된 명언 (일부) ---
1. Albert Einstein: “The world as we have created ... [change, deep-thoughts, thinking, world]
2. J.K. Rowling: “It is our choices, Harry, tha... [abilities, choices]
3. Albert Einstein: “There are only two ways to li... [inspirational, life, live, miracle, miracles]
4. Jane Austen: “The person, be it gentleman o... [aliteracy, books, classic, humor]
5. Marilyn Monroe: “Imperfection is beauty, madne... [be-yourself, inspirational]
...
29. Albert Einstein: “Logic will get you from A to ... [imagination]
30. Bob Marley: “One good thing about music, w... [music]


---
## 5. 수집한 데이터를 Pandas DataFrame으로 변환 및 저장하기 📊💾

지금까지 리스트 형태로 수집한 명언 데이터를 다루기 쉽게 Pandas DataFrame으로 변환할 수 있습니다. DataFrame으로 변환하면 정렬, 필터링, 분석 등이 용이해지고, CSV 파일 등으로 쉽게 저장할 수 있습니다.

** Pandas DataFrame 주요 특징 **
* 2차원 테이블 형태의 자료구조
* 각 열(column)은 서로 다른 타입을 가질 수 있음 (숫자, 문자열, 불리언 등)
* 데이터 분석 및 전처리에 매우 유용

여기서는 수집한 `all_quotes_data` 리스트를 DataFrame으로 만들고, 처음 몇 줄을 확인한 뒤 CSV 파일로 저장하는 예시를 보여줍니다.

In [ ]:
if all_quotes_data: # 데이터가 있을 경우에만 실행
    # 리스트를 Pandas DataFrame으로 변환
    df_quotes = pd.DataFrame(all_quotes_data)

    print("\n--- Pandas DataFrame 변환 결과 (상위 5개) ---")
    print(df_quotes)

    print(f"\nDataFrame 형태: {df_quotes.shape} (행, 열)")

    # CSV 파일로 저장 (index=False는 DataFrame의 인덱스를 파일에 쓰지 않도록 함)
    # Colab 환경에서는 이 파일이 세션 저장 공간에 임시로 저장됩니다.
    # 좌측 파일 탐색기에서 확인할 수 있으며, 로컬로 다운로드 가능합니다.
    csv_filename = 'quotes_scraped.csv'
    df_quotes.to_csv(csv_filename, index=False, encoding='utf-8-sig') # utf-8-sig로 한글 깨짐 방지
    print(f"\n'{csv_filename}' 파일로 저장 완료!")

    # # 파일 내용 간단히 확인 (Colab에서 바로 확인하기 위함)
    # print("\n--- CSV 파일 내용 (처음 5줄) ---")
    # with open(csv_filename, 'r', encoding='utf-8-sig') as f:
    #     for i, line in enumerate(f):
    #         if i < 5:
    #             print(line.strip())
    #         else:
    #             break
else:
    print("\n수집된 데이터가 없어 DataFrame으로 변환할 수 없습니다.")
print("웹 스크래핑 시에는 항상 웹사이트의 robots.txt와 이용 약관을 준수하고, 서버에 과도한 부하를 주지 않도록 주의하세요.")


--- Pandas DataFrame 변환 결과 (상위 5개) ---
                                                 text              author  \
0   “The world as we have created it is a process ...     Albert Einstein   
1   “It is our choices, Harry, that show what we t...        J.K. Rowling   
2   “There are only two ways to live your life. On...     Albert Einstein   
3   “The person, be it gentleman or lady, who has ...         Jane Austen   
4   “Imperfection is beauty, madness is genius and...      Marilyn Monroe   
..                                                ...                 ...   
95  “You never really understand a person until yo...          Harper Lee   
96  “You have to write the book that wants to be w...   Madeleine L'Engle   
97  “Never tell the truth to people who are not wo...          Mark Twain   
98        “A person's a person, no matter how small.”           Dr. Seuss   
99  “... a mind needs books as a sword needs a whe...  George R.R. Martin   

                                   